Get NDA data dictionary items pertaining to NIH Toolbox data and merge with Crosswalk

In [1]:
# Programatically retreive data from the dictionary service for every hcp_variable that has 
# been mapped to the NDA by their data curators for line item comparisons between Toolbox Data Dicitonary and 
# NDA data dictionary


import requests
import json
import pandas as pd

#example to expand
shortname = 'cogcomp01'
varlist=['subjectkey','src_subject_id']
r = requests.get('https://ndar.nih.gov/api/datadictionary/datastructure/{}'
                 .format(shortname),
                  headers={'Accept':'application/json'})
structure = json.loads(r.text)
print(structure.keys())
df=pd.DataFrame(structure['dataElements'])
df=df[['name','description','valueRange','notes']]
dfxwalk=df.loc[df.name.isin(varlist)] 
dfxwalk


dict_keys(['shortName', 'title', 'sources', 'categories', 'dataType', 'status', 'publicStatus', 'publishDate', 'modifiedDate', 'dataElements'])


,name,description,valueRange,notes
0,subjectkey,The NDAR Global Unique Identifier (GUID) for r...,NDAR*,None
1,src_subject_id,Subject ID how it's defined in lab/project,None,None


In [2]:
#read crosswalkfile
pathin="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/"
crosswalkfile="Crosswalk_NIH_Toolbox_2_NDA.csv"
crosswalk=pd.read_csv(crosswalkfile,header=0,low_memory=False, encoding = "ISO-8859-1")
crosswalk.head()


,validated,Measurement System,Domain,Item ID,Stem,Context,DataType,Responses,Inst,hcp_variable,...,hcp_variable_upload,nda_structure,nda_element,template,inst_short,Source,description,valueRange,notes,nda_structure_link
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00095,...,nih_tlbx_rawscore,tlbx_fearanx01,nih_tlbx_rawscore,NaN,Anxiety_Summary_3-7,NaN,RawScore,NaN,NaN,https://nda.nih.gov/data_structure.html?short_...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00096,...,nih_tlbx_theta,tlbx_fearanx01,nih_tlbx_theta,NaN,Anxiety_Summary_3-7,NaN,Theta,NaN,999=NA/NK/missing,https://nda.nih.gov/data_structure.html?short_...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00097,...,nih_tlbx_tscore,tlbx_fearanx01,nih_tlbx_tscore,NaN,Anxiety_Summary_3-7,NaN,T-score,0::200; 999,999=NA/NK/missing,https://nda.nih.gov/data_structure.html?short_...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00098,...,nih_tlbx_se,tlbx_fearanx01,nih_tlbx_se,NaN,Anxiety_Summary_3-7,NaN,Standard Error,NaN,999=NA/NK/missing,https://nda.nih.gov/data_structure.html?short_...
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00099,...,nih_tlbx_agegencsc,tlbx_fearanx01,nih_tlbx_agegencsc,NaN,Anxiety_Summary_3-7,NaN,Age-Gender-Corrected T-score,0::250; 999,999=NA/NK/missing,https://nda.nih.gov/data_structure.html?short_...


In [3]:
#for a given structure (shortname), grab all the metadata for a list of elements as a dataframe
def getNDAdetails(structure_name=shortname,crosswalk=crosswalk):
    varlist=crosswalk.loc[crosswalk.nda_structure==structure_name,'nda_element'].to_list()
    r = requests.get('https://ndar.nih.gov/api/datadictionary/datastructure/{}'
                 .format(structure_name),
                  headers={'Accept':'application/json'})
    structure = json.loads(r.text)
    df=pd.DataFrame(structure['dataElements'])
    df2=df[['name','description','valueRange','notes']].copy()
    dfxwalk=df2.loc[df2.name.isin(varlist)].copy() 
    dfxwalk['nda_structure']=structure_name
    dfxwalk=dfxwalk.rename(columns={'name':'nda_element'})
    return dfxwalk



In [4]:
#test
#dfxwalk=getNDAdetails(structure_name='prang01')
#dfxwalk
#dfxwalk=getNDAdetails(structure_name='tlbx_wellbeing01')
#dfxwalk
dfxwalk=getNDAdetails(structure_name='tlbx_motor01')
dfxwalk

,nda_element,description,valueRange,notes,nda_structure
5,version_form,Form used/assessment name,None,None,tlbx_motor01
8,en6m50_2m_10ft_marks,Endurance - 2 minute walk - Number of 10 ft marks,None,None,tlbx_motor01
9,en6m50_2m_ft,Endurance - 2 minute walk - Feet,None,None,tlbx_motor01
10,en6m50_2m_inches,Endurance - 2 minute walk - Inches,None,None,tlbx_motor01
11,en6m50_2m_turns,Endurance - 2 minute walk - Number of course t...,None,None,tlbx_motor01
13,end_2m_standardsc,Endurance -Normative Standard Score,None,None,tlbx_motor01
19,grip_standardsc_dom,9-Hole Grip Strength Dominant Hand Normative S...,None,None,tlbx_motor01
20,grip_standardsc_nondom,9-Hole Grip Strength Non-Dominant Hand Normati...,None,None,tlbx_motor01
25,lo4mt1,Locomotion - 4 meter walk - Trial 1 (Usual Pac...,None,None,tlbx_motor01
26,lo4mt2,Locomotion - 4 meter walk - Trial 2 (Fast Pace...,None,None,tlbx_motor01


In [5]:
#append all this information together for a merge
allmetadata=pd.DataFrame()
for structshort in crosswalk.nda_structure.unique():
    try:
        dfxwalk=getNDAdetails(structure_name=structshort)
        allmetadata=pd.concat([allmetadata,dfxwalk])
    except:
        print('could not process {}'.format(structshort))
 



could not process nan


In [6]:
allmetadata.head()

,nda_element,description,valueRange,notes,nda_structure
5,primary_language,Subject's Primary Language,None,None,tlbx_fearanx01
7,version_form,Form used/assessment name,None,None,tlbx_fearanx01
8,pedrepanx50,In the past 7 days I worried when I was at home,1::5,1=Never; 2=Almost Never; 3=Sometimes; 4=Often;...,tlbx_fearanx01
9,pedrepanx51,In the past 7 days I got scared really easily,1::5,1=Never; 2=Almost Never; 3=Sometimes; 4=Often;...,tlbx_fearanx01
10,pedproxanx29,Please indicate how often or true the behavior...,0::2,0=Never or not true; 1=Sometimes or somewhat t...,tlbx_fearanx01


In [6]:
#hcp_variable_upload = mda_ement in the CROSSWALK is supposed to be the same as nda_element (we have found typos from the NDA curators, which is understandable, but needs to be checked)
#merge the annotation by 'hcp_variable_upload' to see if any NDA descriptors are missing.
issues=pd.merge(crosswalk.drop(columns=['description','valueRange','notes']),allmetadata,how='left',on=['nda_structure','hcp_variable_upload'])


KeyError: 'hcp_variable_upload'

In [7]:
#now merge this data into the crosswalk and write the new csv
newcrosswalk=pd.merge(crosswalk.drop(columns=['description','valueRange','notes']),allmetadata,how='left',on=['nda_structure','nda_element'])

In [8]:
newcrosswalk.head()

,validated,Measurement System,Domain,Item ID,Stem,Context,DataType,Responses,Inst,hcp_variable,...,specialty_code,hcp_variable_upload,nda_structure,nda_element,template,inst_short,Source,description,valueRange,notes
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00095,...,NaN,nih_tlbx_rawscore,tlbx_fearanx01,nih_tlbx_rawscore,NaN,Anxiety_Summary_3-7,NaN,RawScore,None,None
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00096,...,NaN,nih_tlbx_theta,tlbx_fearanx01,nih_tlbx_theta,NaN,Anxiety_Summary_3-7,NaN,Theta,None,999=NA/NK/missing
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00097,...,NaN,nih_tlbx_tscore,tlbx_fearanx01,nih_tlbx_tscore,NaN,Anxiety_Summary_3-7,NaN,T-score,0::200; 999,999=NA/NK/missing
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00098,...,NaN,nih_tlbx_se,tlbx_fearanx01,nih_tlbx_se,NaN,Anxiety_Summary_3-7,NaN,Standard Error,None,999=NA/NK/missing
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00099,...,NaN,nih_tlbx_agegencsc,tlbx_fearanx01,nih_tlbx_agegencsc,NaN,Anxiety_Summary_3-7,NaN,Age-Gender-Corrected T-score,0::250; 999,999=NA/NK/missing


In [9]:
newcrosswalk.to_csv(pathin+crosswalkfile,index=False)#crosswalkfile,index=False)

In [10]:
newcrosswalk.head()

,validated,Measurement System,Domain,Item ID,Stem,Context,DataType,Responses,Inst,hcp_variable,...,specialty_code,hcp_variable_upload,nda_structure,nda_element,template,inst_short,Source,description,valueRange,notes
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00095,...,NaN,nih_tlbx_rawscore,tlbx_fearanx01,nih_tlbx_rawscore,NaN,Anxiety_Summary_3-7,NaN,RawScore,None,None
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00096,...,NaN,nih_tlbx_theta,tlbx_fearanx01,nih_tlbx_theta,NaN,Anxiety_Summary_3-7,NaN,Theta,None,999=NA/NK/missing
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00097,...,NaN,nih_tlbx_tscore,tlbx_fearanx01,nih_tlbx_tscore,NaN,Anxiety_Summary_3-7,NaN,T-score,0::200; 999,999=NA/NK/missing
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00098,...,NaN,nih_tlbx_se,tlbx_fearanx01,nih_tlbx_se,NaN,Anxiety_Summary_3-7,NaN,Standard Error,None,999=NA/NK/missing
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anxiety Summary Parent Report (3-7),dummy_00099,...,NaN,nih_tlbx_agegencsc,tlbx_fearanx01,nih_tlbx_agegencsc,NaN,Anxiety_Summary_3-7,NaN,Age-Gender-Corrected T-score,0::250; 999,999=NA/NK/missing
